In [1]:
from pydantic import BaseModel
import requests
import json
import tomllib as tl

with open("config.toml", "rb") as f:
    config = tl.load(f)
    ssm = config["SSMERP"]

class Service:
    base_url: str = "https://preprod-api.myinvois.hasil.gov.my"

class LoginSchema(BaseModel):
    client_id: str
    client_secret: str
    grant_type: str = "client_credentials"
    scope: str = "InvoicingAPI"

class LoginResponse(BaseModel):
    access_token: str
    token_type: str
    expires_in: int
    scope: str

class LoginService(Service):
    @classmethod
    def login(self, body: LoginSchema) -> LoginResponse:
        headers = {
            "Content-Type": "application/x-www-form-urlencoded"
        }
        res = requests.post(self.base_url + "/connect/token", data=body.model_dump(), headers=headers)
        return LoginResponse(
            **json.loads(res.text)
        )

body = LoginSchema(
    client_id=ssm["ClientID"],
    client_secret=ssm["Secret1"],
)
token = LoginService.login(body)
token.model_dump()

{'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6Ijk2RjNBNjU2OEFEQzY0MzZDNjVBNDg1MUQ5REM0NTlFQTlCM0I1NTRSUzI1NiIsIng1dCI6Imx2T21Wb3JjWkRiR1draFIyZHhGbnFtenRWUSIsInR5cCI6ImF0K2p3dCJ9.eyJpc3MiOiJodHRwczovL3ByZXByb2QtaWRlbnRpdHkubXlpbnZvaXMuaGFzaWwuZ292Lm15IiwibmJmIjoxNzQyNTIxODc0LCJpYXQiOjE3NDI1MjE4NzQsImV4cCI6MTc0MjUyNTQ3NCwiYXVkIjpbIkludm9pY2luZ0FQSSIsImh0dHBzOi8vcHJlcHJvZC1pZGVudGl0eS5teWludm9pcy5oYXNpbC5nb3YubXkvcmVzb3VyY2VzIl0sInNjb3BlIjpbIkludm9pY2luZ0FQSSJdLCJjbGllbnRfaWQiOiI2MDE3ZDJkNC1hYmVlLTRkNzctYmM0ZS0xM2M1NTAzY2NkN2EiLCJJc1RheFJlcHJlcyI6IjEiLCJJc0ludGVybWVkaWFyeSI6IjAiLCJJbnRlcm1lZElkIjoiMCIsIkludGVybWVkVElOIjoiIiwiSW50ZXJtZWRST0IiOiIiLCJJbnRlcm1lZEVuZm9yY2VkIjoiMiIsIm5hbWUiOiJEMjY0MDQ1NjkwNjA6NjAxN2QyZDQtYWJlZS00ZDc3LWJjNGUtMTNjNTUwM2NjZDdhIiwiU1NJZCI6IjZhZmVlMDJiLTRiMzgtN2EzMS0xNzZlLWY2YWEyODhmYjRlZCIsInByZWZlcnJlZF91c2VybmFtZSI6IkRldmVsb3BtZW50IiwiVGF4SWQiOiI1MjAwMyIsIlRheHBheWVyVElOIjoiRDI2NDA0NTY5MDYwIiwiUHJvZklkIjoiNjkxNTAiLCJJc1RheEFkbWluIjoiMCIsIklzU3lzdGVtIjoiMSJ9.Rxx9

In [2]:
url = "/api/v1.0/documenttypes"
headers = {
    "Authorization": f"{token.token_type} {token.access_token}"
}

res = requests.get(Service.base_url + url, headers=headers)
import pprint
pprint.pprint(json.loads(res.text))

{'result': [{'activeFrom': '2024-05-02T10:34:11.15Z',
             'activeTo': None,
             'description': 'Invoice',
             'documentTypeVersions': [{'activeFrom': '2024-04-26T17:47:49.2Z',
                                       'activeTo': None,
                                       'description': 'Version 1',
                                       'id': 1,
                                       'name': 'Version 1',
                                       'status': 'Published',
                                       'versionNumber': '1.0'},
                                      {'activeFrom': '2024-06-21T15:13:17.29Z',
                                       'activeTo': None,
                                       'description': 'Version 2',
                                       'id': 2,
                                       'name': 'Version 2',
                                       'status': 'Published',
                                       'versionNumber': '1.1'}],


In [3]:
url = "/api/v1.0/documenttypes/1"
headers = {
    "Authorization": f"{token.token_type} {token.access_token}"
}

res = requests.get(Service.base_url + url, headers=headers)
import pprint
pprint.pprint(json.loads(res.text))

{'activeFrom': '2024-05-02T10:34:11.15Z',
 'activeTo': None,
 'description': 'Invoice',
 'documentTypeVersions': [{'activeFrom': '2024-04-26T17:47:49.2Z',
                           'activeTo': None,
                           'description': 'Version 1',
                           'id': 1,
                           'name': 'Version 1',
                           'status': 'Published',
                           'versionNumber': '1.0'},
                          {'activeFrom': '2024-06-21T15:13:17.29Z',
                           'activeTo': None,
                           'description': 'Version 2',
                           'id': 2,
                           'name': 'Version 2',
                           'status': 'Published',
                           'versionNumber': '1.1'}],
 'id': 1,
 'invoiceTypeCode': '01',
 'workflowParameters': [{'activeFrom': '2024-03-14T04:59:24.076Z',
                         'activeTo': None,
                         'id': 1,
                        

In [4]:
from pydantic import BaseModel, Field
from typing import List, Any
import datetime

class ValueStr_(BaseModel):
    value: str = Field(..., alias="_")
class ValueInt_(BaseModel):
    value: int = Field(..., alias="_")
class ValueFloat_(BaseModel):
    value: float = Field(..., alias="_")
class ValueBool_(BaseModel):
    value: bool = Field(..., alias="_")
class ValueDate_(BaseModel):
    value: datetime.date = Field(..., alias="_")
class ValueDateTime_(BaseModel):
    value: datetime.datetime = Field(..., alias="_")
class ValueTime_(BaseModel):
    value: datetime.time = Field(..., alias="_")


class InvoiceTypeCode_(BaseModel):
    value: str = Field(default="01", alias="_")
    listVersionID: str = "1.0"
class DocumentCurrencyCode_(BaseModel):
    value: str = Field(default="MYR", alias="_")
class TaxCurrencyCode_(BaseModel):
    value: str = Field(default="MYR", alias="_")
class InvoicePeriod_(BaseModel):
    StartDate: List[ValueDate_]
    EndDate: List[ValueDate_]
    Description: List[ValueStr_]

class Invoice(BaseModel):
    ID: List[ValueStr_]
    IssueDate: List[ValueDate_]
    IssueTime: List[ValueTime_]
    InvoiceTypeCode: List[InvoiceTypeCode_]
    DocumentCurrencyCode: List[DocumentCurrencyCode_]
    TaxCurrencyCode: List[TaxCurrencyCode_]
    InvoicePeriod: List[InvoicePeriod_]

Invoice(
    **{
        "ID": [{"_": "INV001"}],
        "IssueDate": [{"_": "2021-01-01"}],
        "IssueTime": [{"_": "12:00:00"}],
        "InvoiceTypeCode": [{"_": "01"}],
        "DocumentCurrencyCode": [{"_": "MYR"}],
        "TaxCurrencyCode": [{"_": "MYR"}],
        "InvoicePeriod": [{
            "StartDate": [{"_": "2021-01-01"}],
            "EndDate": [{"_": "2021-01-31"}],
            "Description": [{"_": "January 2021"}]
        }]
    }
).model_dump(by_alias=True, mode="json")

{'ID': [{'_': 'INV001'}],
 'IssueDate': [{'_': '2021-01-01'}],
 'IssueTime': [{'_': '12:00:00'}],
 'InvoiceTypeCode': [{'_': '01', 'listVersionID': '1.0'}],
 'DocumentCurrencyCode': [{'_': 'MYR'}],
 'TaxCurrencyCode': [{'_': 'MYR'}],
 'InvoicePeriod': [{'StartDate': [{'_': '2021-01-01'}],
   'EndDate': [{'_': '2021-01-31'}],
   'Description': [{'_': 'January 2021'}]}]}

In [17]:
from pydantic import BaseModel, Field
from typing import List, Any
import datetime

class ValueStr_(BaseModel):
    value: str = Field(..., alias="_")
class ValueInt_(BaseModel):
    value: int = Field(..., alias="_")
class ValueFloat_(BaseModel):
    value: float = Field(..., alias="_")
class ValueBool_(BaseModel):
    value: bool = Field(..., alias="_")
class ValueDate_(BaseModel):
    value: datetime.date = Field(..., alias="_")
class ValueDateTime_(BaseModel):
    value: datetime.datetime = Field(..., alias="_")
class ValueTime_(BaseModel):
    value: datetime.time = Field(..., alias="_")


class AdditionalAccountID_(ValueStr_):
    schemeAgencyName: str = "CertEX"
class IndustryClassificationCode_(ValueStr_):
    name: str
class PartyIdentificationDetails_(ValueStr_):
    schemeID: str
class PartyIdentification_(BaseModel):
    ID: List[PartyIdentificationDetails_]
class AddressLine_(BaseModel):
    Line: List[ValueStr_]
class IdentificationCode_(ValueStr_):
    value: str = Field("MYS", alias="_")
    listID: str = "ISO3166-1"
    listAgencyID: str = "6"
class Country_(BaseModel):
    IdentificationCode: List[IdentificationCode_]

class PostalAddress_(BaseModel):
    CityName: List[ValueStr_]
    PostalZone: List[ValueStr_]
    CountrySubentityCode: List[ValueStr_]
    AddressLine: List[AddressLine_]
    Country: List[Country_]


class PartyLegalEntity_(BaseModel):
    RegistrationName: List[ValueStr_] = Field(..., alias="RegistrationName")

class Contact_(BaseModel):
    Telephone: List[ValueStr_]
    ElectronicMail: List[ValueStr_]


class Party_(BaseModel):
    AdditionalAccountID: List[AdditionalAccountID_]
    PostalAddress: List[PostalAddress_]
    PartyIdentification: List[PartyIdentification_]
    PartyLegalEntity: List[PartyLegalEntity_]
    Contact: List[Contact_]


addr = PostalAddress_(
    **{
        "CityName": [{"_": "Kuala Lumpur"}],
        "PostalZone": [{"_": "50480"}],
        "CountrySubentityCode": [{"_": "10"}],
        "AddressLine": [
            {"Line": [{"_": "Lot 66"}]},
            {"Line": [{"_": "Bangunan Merdeka"}]},
            {"Line": [{"_": "Persiaran Jaya"}]}
        ],
        "Country": [
            {"IdentificationCode": [{"_": "MYS"}]}
        ],
    }
) #.model_dump(by_alias=True, mode="json")

from pprint import pprint
pprint(Party_(
    **{
        "AdditionalAccountID": [{"_": "1234567890"}],
        "PostalAddress": [{
        "CityName": [{"_": "Kuala Lumpur"}],
        "PostalZone": [{"_": "50480"}],
        "CountrySubentityCode": [{"_": "10"}],
        "AddressLine": [
            {"Line": [{"_": "Lot 66"}]},
            {"Line": [{"_": "Bangunan Merdeka"}]},
            {"Line": [{"_": "Persiaran Jaya"}]}
        ],
        "Country": [
            {"IdentificationCode": [{"_": "MYS"}]}
        ],
    }],
        "PartyIdentification": [{"ID": [{"_": "1234567890", "schemeID": "MYID"}]}],
        "PartyLegalEntity": [{"RegistrationName": [{"_": "My Company Sdn Bhd"}]}],
        "Contact": [
            {"Telephone": [{"_": "0123456789"}],
            "ElectronicMail":  [{"_": "supplier@email.com"}]}
        ]
    }
).model_dump(by_alias=True, mode="json"))

{'AdditionalAccountID': [{'_': '1234567890', 'schemeAgencyName': 'CertEX'}],
 'Contact': [{'ElectronicMail': [{'_': 'supplier@email.com'}],
              'Telephone': [{'_': '0123456789'}]}],
 'PartyIdentification': [{'ID': [{'_': '1234567890', 'schemeID': 'MYID'}]}],
 'PartyLegalEntity': [{'RegistrationName': [{'_': 'My Company Sdn Bhd'}]}],
 'PostalAddress': [{'AddressLine': [{'Line': [{'_': 'Lot 66'}]},
                                    {'Line': [{'_': 'Bangunan Merdeka'}]},
                                    {'Line': [{'_': 'Persiaran Jaya'}]}],
                    'CityName': [{'_': 'Kuala Lumpur'}],
                    'Country': [{'IdentificationCode': [{'_': 'MYS',
                                                         'listAgencyID': '6',
                                                         'listID': 'ISO3166-1'}]}],
                    'CountrySubentityCode': [{'_': '10'}],
                    'PostalZone': [{'_': '50480'}]}]}


In [20]:
from enum import Enum

'''
Overview
Payment Mode that may be selected at the time of document submission.

#List
Code	Description
01	Cash
02	Cheque
03	Bank Transfer
04	Credit Card
05	Debit Card
06	e-Wallet / Digital Wallet
07	Digital Bank
08	Others
'''
class PaymentModeCode(str, Enum):
    Cash = "01"
    Cheque = "02"
    BankTransfer = "03"
    CreditCard = "04"
    DebitCard = "05"
    EWallet = "06"
    DigitalBank = "07"
    Others = "08"

class PaymentMeans_(BaseModel):
    PaymentMeansCode: List[ValueStr_]
    
class PaymentTerms_(BaseModel):
    Note: List[ValueStr_]
    
# PaymentMeans_(
#     **{
#         "PaymentMeansCode": [{"_": "03"}]
#     }
# ).model_dump(by_alias=True, mode="json")
PaymentTerms_(
    **{
        "Note": [{"_": "Payment due in 30 days"}]
    }
).model_dump(by_alias=True, mode="json")

{'Note': [{'_': 'Payment due in 30 days'}]}

In [ ]:
'''Overview
Tax types are used when entering tax information as part of the invoice (where applicable).

#List
#Taxable Types
Code	Description
01	Sales Tax
02	Service Tax
03	Tourism Tax
04	High-Value Goods Tax
05	Sales Tax on Low Value Goods
06	Not Applicable
E	Tax exemption (where applicable)'''
from enum import Enum

class TaxTypeCode(str, Enum):
    SalesTax = "01"
    ServiceTax = "02"
    TourismTax = "03"
    HighValueGoodsTax = "04"
    SalesTaxLowValueGoods = "05"
    NotApplicable = "06"
    TaxExemption = "E"
    
class TaxAmount_(ValueFloat_):
    currencyID: str = "MYR"

class TaxSchemeID_(ValueStr_):
    value: str = Field(default="01", alias="_")
    schemeID: str = "UN/ECE 5153"
    schemeAgencyID: str = "6"
class TaxScheme_(BaseModel):
    ID: List[TaxSchemeID_]
class TaxCategory_(BaseModel):
    ID: List[ValueStr_]
    TaxScheme: List[TaxScheme_]

    
class TaxSubtotal_(BaseModel):
    TaxableAmount: List[TaxAmount_]
    TaxAmount: List[TaxAmount_]
    TaxCategory: List[TaxCategory_]

class TaxTotal_(BaseModel):
    TaxAmount: List[TaxAmount_]
    TaxSubtotal: List[TaxSubtotal_]

# TaxTotal_(
#     **{
#         "TaxAmount": [{"_": 87.63, "currencyID": "MYR"}],
#         "TaxSubtotal": [{
#             "TaxableAmount": [{"_": 87.63, "currencyID": "MYR"}],
#             "TaxAmount": [{"_": 87.63, "currencyID": "MYR"}],
#             "TaxCategory": [{
#                 "ID": [{"_": "01"}],
#                 "TaxScheme": [{
#                     "ID": [{"_": "OTH", "schemeID": "UN/ECE 5153", "schemeAgencyID": "6"}]
#                 }]
#             }]
#         }]
#     }
# ).model_dump(by_alias=True, mode="json")

class LegalMonetaryTotal_(BaseModel):
    # mandatory fields
    TaxExclusiveAmount: List[TaxAmount_]
    TaxInclusiveAmount: List[TaxAmount_]
    PayableAmount: List[TaxAmount_]
    
    # optional fields
    LineExtensionAmount: List[TaxAmount_] | None = None
    AllowanceTotalAmount: List[TaxAmount_] | None = None
    ChargeTotalAmount: List[TaxAmount_] | None = None
    
LegalMonetaryTotal_(
    **{
        "TaxExclusiveAmount": [{"_": 87.63, "currencyID": "MYR"}],
        "TaxInclusiveAmount": [{"_": 87.63, "currencyID": "MYR"}],
        "PayableAmount": [{"_": 87.63, "currencyID": "MYR"}],
    }
).model_dump(by_alias=True, mode="json", exclude_none=True)
    

{'TaxExclusiveAmount': [{'_': 87.63, 'currencyID': 'MYR'}],
 'TaxInclusiveAmount': [{'_': 87.63, 'currencyID': 'MYR'}],
 'PayableAmount': [{'_': 87.63, 'currencyID': 'MYR'}]}

In [39]:
'''
Overview
Classification code list defines the category of products or services being billed as a result of a commercial transaction.

#List
Code	Description
001	Breastfeeding equipment
002	Child care centres and kindergartens fees
003	Computer, smartphone or tablet
004	Consolidated e-Invoice
005	Construction materials (as specified under Fourth Schedule of the Lembaga Pembangunan Industri Pembinaan Malaysia Act 1994)
006	Disbursement
007	Donation
008	e-Commerce - e-Invoice to buyer / purchaser
009	e-Commerce - Self-billed e-Invoice to seller, logistics, etc.
010	Education fees
011	Goods on consignment (Consignor)
012	Goods on consignment (Consignee)
013	Gym membership
014	Insurance - Education and medical benefits
015	Insurance - Takaful or life insurance
016	Interest and financing expenses
017	Internet subscription
018	Land and building
019	Medical examination for learning disabilities and early intervention or rehabilitation treatments of learning disabilities
020	Medical examination or vaccination expenses
021	Medical expenses for serious diseases
022	Others
023	Petroleum operations (as defined in Petroleum (Income Tax) Act 1967)
024	Private retirement scheme or deferred annuity scheme
025	Motor vehicle
026	Subscription of books / journals / magazines / newspapers / other similar publications
027	Reimbursement
028	Rental of motor vehicle
029	EV charging facilities (Installation, rental, sale / purchase or subscription fees)
030	Repair and maintenance
031	Research and development
032	Foreign income
033	Self-billed - Betting and gaming
034	Self-billed - Importation of goods
035	Self-billed - Importation of services
036	Self-billed - Others
037	Self-billed - Monetary payment to agents, dealers or distributors
038	Sports equipment, rental / entry fees for sports facilities, registration in sports competition or sports training fees imposed by associations / sports clubs / companies registered with the Sports Commissioner or Companies Commission of Malaysia and carrying out sports activities as listed under the Sports Development Act 1997
039	Supporting equipment for disabled person
040	Voluntary contribution to approved provident fund
041	Dental examination or treatment
042	Fertility treatment
043	Treatment and home care nursing, daycare centres and residential care centers
044	Vouchers, gift cards, loyalty points, etc
045	Self-billed - Non-monetary payment to agents, dealers or distributors
'''
from enum import Enum

class ItemClassificationCode(str, Enum):
    BreastfeedingEquipment = "001"
    ChildCareCentresKindergartensFees = "002"
    ComputerSmartphoneTablet = "003"
    ConsolidatedEInvoice = "004"
    ConstructionMaterials = "005"
    Disbursement = "006"
    Donation = "007"
    ECommerceEInvoiceBuyerPurchaser = "008"
    ECommerceSelfBilled = "009"
    EducationFees = "010"
    GoodsOnConsignmentConsignor = "011"
    GoodsOnConsignmentConsignee = "012"
    GymMembership = "013"
    InsuranceEducationMedicalBenefits = "014"
    InsuranceTakafulLifeInsurance = "015"
    InterestFinancingExpenses = "016"
    InternetSubscription = "017"
    LandBuilding = "018"
    MedicalExaminationLearning = "019"
    MedicalExaminationVaccinationExpenses = "020"
    MedicalExpensesSeriousDiseases = "021"
    Others = "022"
    PetroleumOperations = "023"
    PrivateRetirementSchemeDeferredAnnuityScheme = "024"
    MotorVehicle = "025"
    SubscriptionBooks = "026"
    Reimbursement = "027"
    RentalMotorVehicle = "028"
    EVChargingFacilities = "029"
    RepairMaintenance = "030"
    ResearchDevelopment = "031"
    ForeignIncome = "032"
    SelfBilledBettingGaming = "033"
    SelfBilledImportationGoods = "034"
    SelfBilledImportationServices = "035"
    SelfBilledOthers = "036"
    SelfBilledMonetaryPayment = "037"
    SportsEquipmentRentalEntryFees = "038"
    SupportingEquipmentDisabledPerson = "039"
    VoluntaryContributionApprovedProvidentFund = "040"
    DentalExaminationTreatment = "041"
    FertilityTreatment = "042"
    TreatmentHomeCareNursing = "043"
    VouchersGiftCardsLoyaltyPoints = "044"
    SelfBilledNonMonetaryPayment = "045"
    
class ItemClassificationDesc(str, Enum):
    BreastfeedingEquipment = "Breastfeeding equipment"
    ChildCareCentresKindergartensFees = "Child care centres and kindergartens fees"
    ComputerSmartphoneTablet = "Computer, smartphone or tablet"
    ConsolidatedEInvoice = "Consolidated e-Invoice"
    ConstructionMaterials = "Construction materials (as specified under Fourth Schedule of the Lembaga Pembangunan Industri Pembinaan Malaysia Act 1994)"
    Disbursement = "Disbursement"
    Donation = "Donation"
    ECommerceEInvoiceBuyerPurchaser = "e-Commerce - e-Invoice to buyer / purchaser"
    ECommerceSelfBilled = "e-Commerce - Self-billed e-Invoice to seller, logistics, etc."
    EducationFees = "Education fees"
    GoodsOnConsignmentConsignor = "Goods on consignment (Consignor)"
    GoodsOnConsignmentConsignee = "Goods on consignment (Consignee)"
    GymMembership = "Gym membership"
    InsuranceEducationMedicalBenefits = "Insurance - Education and medical benefits"
    InsuranceTakafulLifeInsurance = "Insurance - Takaful or life insurance"
    InterestFinancingExpenses = "Interest and financing expenses"
    InternetSubscription = "Internet subscription"
    LandBuilding = "Land and building"
    MedicalExaminationLearning = "Medical examination for learning disabilities and early intervention or rehabilitation treatments of learning disabilities"
    MedicalExaminationVaccinationExpenses = "Medical examination or vaccination expenses"
    MedicalExpensesSeriousDiseases = "Medical expenses for serious diseases"
    Others = "Others"
    PetroleumOperations = "Petroleum operations (as defined in Petroleum (Income Tax) Act 1967)"
    PrivateRetirementSchemeDeferredAnnuityScheme = "Private retirement scheme or deferred annuity scheme"
    MotorVehicle = "Motor vehicle"
    SubscriptionBooks = "Subscription of books / journals / magazines / newspapers / other similar publications"
    Reimbursement = "Reimbursement"
    RentalMotorVehicle = "Rental of motor vehicle"
    EVChargingFacilities = "EV charging facilities (Installation, rental, sale / purchase or subscription fees)"
    RepairMaintenance = "Repair and maintenance"
    ResearchDevelopment = "Research and development"
    ForeignIncome = "Foreign income"
    SelfBilledBettingGaming = "Self-billed - Bettin and gaming"
    SelfBilledImportationGoods = "Self-billed - Importation of goods"
    SelfBilledImportationServices = "Self-billed - Importation of services"
    SelfBilledOthers = "Self-billed - Others"
    SelfBilledMonetaryPayment = "Self-billed - Monetary payment to agents, dealers or distributors"
    SportsEquipmentRentalEntryFees = "Sports equipment, rental / entry fees for sports facilities, registration in sports competition or sports training fees imposed by associations / sports clubs / companies registered with the Sports Commissioner or Companies Commission of Malaysia and carrying out sports activities as listed under the Sports Development Act 1997"
    SupportingEquipmentDisabledPerson = "Supporting equipment for disabled person"
    VoluntaryContributionApprovedProvidentFund = "Voluntary contribution to approved provident fund"
    DentalExaminationTreatment = "Dental examination or treatment"
    FertilityTreatment = "Fertility treatment"
    TreatmentHomeCareNursing = "Treatment and home care nursing, daycare centres and residential care centers"
    VouchersGiftCardsLoyaltyPoints = "Vouchers, gift cards, loyalty points, etc"
    SelfBilledNonMonetaryPayment = "Self-billed - Non-monetary payment to agents, dealers or distributors"
    
class UnitType(str, Enum):
    tonne = "TNE"

class InvoicedQuantity_(ValueFloat_):
    unitCode: str = "TNE"
class Currency_(ValueFloat_):
    currencyID: str = "MYR"
class ItemClassificationCode_(ValueStr_):
    value: str = Field(ItemClassificationCode.Others.value, alias="_")
    listID: str = "CLASS"
class CommodityClassification_(BaseModel):
    ItemClassificationCode: List[ItemClassificationCode_]
    
class Item_(BaseModel):
    CommodityClassification: List[CommodityClassification_]
    Description: List[ValueStr_]
    
class Price_(BaseModel):
    PriceAmount: List[Currency_]
class ItemPriceExtension_(BaseModel):
    Amount: List[Currency_]

    
class InvoiceLine_(BaseModel):
    ID: List[ValueInt_] # ID of the line item
    Item: List[Item_] # Item classification code and description
    Price: List[Price_] # Price of the item
    InvoicedQuantity: List[InvoicedQuantity_] # Quantity of the item
    TaxTotal: List[TaxTotal_] # Tax total of the item
    ItemPriceExtension: List[ItemPriceExtension_] # Subtotal of the item
    LineExtensionAmount: List[Currency_] # Total Excluding Tax
    
    
InvoiceLine_(
    **{
        "ID": [{"_": 1}],
        "Item": [{
            "CommodityClassification": [{
                "ItemClassificationCode": [{"_": ItemClassificationCode.Others.value}]
            }],
            "Description": [{"_": "Item Description"}]
        }],
        "Price": [{
            "PriceAmount": [{"_": 100.00, "currencyID": "MYR"}]
        }],
        "InvoicedQuantity": [{
            "_": 1.0,
            "unitCode": "TNE"
        }],
        "TaxTotal": [{
            "TaxAmount": [{"_": 6.00, "currencyID": "MYR"}],
            "TaxSubtotal": [{
                "TaxableAmount": [{"_": 100.00, "currencyID": "MYR"}],
                "TaxAmount": [{"_": 6.00, "currencyID": "MYR"}],
                "TaxCategory": [{
                    "ID": [{"_": "01"}],
                    "TaxScheme": [{
                        "ID": [{"_": "OTH", "schemeID": "UN/ECE 5153", "schemeAgencyID": "6"}]
                    }]
                }]
            }]
        }],
        "ItemPriceExtension": [{
            "Amount": [{"_": 100.00, "currencyID": "MYR"}]
        }],
        "LineExtensionAmount": [{"_": 100.00, "currencyID": "MYR"}]
    }
).model_dump(by_alias=True, mode="json", exclude_none=True)


{'ID': [{'_': 1}],
 'Item': [{'CommodityClassification': [{'ItemClassificationCode': [{'_': '022',
       'listID': 'CLASS'}]}],
   'Description': [{'_': 'Item Description'}]}],
 'Price': [{'PriceAmount': [{'_': 100.0, 'currencyID': 'MYR'}]}],
 'InvoicedQuantity': [{'_': 1.0, 'unitCode': 'TNE'}],
 'TaxTotal': [{'TaxAmount': [{'_': 6.0, 'currencyID': 'MYR'}],
   'TaxSubtotal': [{'TaxableAmount': [{'_': 100.0, 'currencyID': 'MYR'}],
     'TaxAmount': [{'_': 6.0, 'currencyID': 'MYR'}],
     'TaxCategory': [{'ID': [{'_': '01'}],
       'TaxScheme': [{'ID': [{'_': 'OTH',
           'schemeID': 'UN/ECE 5153',
           'schemeAgencyID': '6'}]}]}]}]}],
 'ItemPriceExtension': [{'Amount': [{'_': 100.0, 'currencyID': 'MYR'}]}],
 'LineExtensionAmount': [{'_': 100.0, 'currencyID': 'MYR'}]}

In [ ]:
class Invoice(BaseModel):
    ID: List[ValueStr_]
    IssueDate: List[ValueDate_]
    IssueTime: List[ValueTime_]
    InvoiceTypeCode: List[InvoiceTypeCode_]
    DocumentCurrencyCode: List[DocumentCurrencyCode_]
    TaxCurrencyCode: List[TaxCurrencyCode_]
    InvoicePeriod: List[InvoicePeriod_]


In [ ]:
idType = "NRIC"
idValue = "730619145970"
url = f"/api/v1.0/taxpayer/search/tin?idType={idType}&idValue={idValue}"
headers = {
    "Authorization": f"{token.token_type} {token.access_token}"
}

res = requests.get(Service.base_url + url, headers=headers)
res.text

'{"tin":"IG11837380030"}'

## Source
1. [JSON Invoice Example](https://sdk.myinvois.hasil.gov.my/files/sdksamples/1.0-Invoice-Sample.json)
2. [Invoice 1.0](https://sdk.myinvois.hasil.gov.my/documents/invoice-v1-0/)
3. [API Documentation](https://sdk.myinvois.hasil.gov.my/einvoicingapi/)